In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import zscore

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, RepeatVector, TimeDistributed, Dense
from tensorflow.keras import backend as K

In [ ]:
df_hist = pd.read_csv('../artifacts/dataset/historical_preprocessed_data.csv')
df_hist

# 1.0 Anomaly Detection

## 1.1 Z-Score

In [ ]:
# Calculate the Z-scores for each feature
z_scores = df_hist[columns_to_apply].apply(zscore)

# Set a threshold for anomaly detection
threshold = 3

# Flag data points where the absolute Z-score is greater than the threshold
anomalies = (z_scores.abs() > threshold)

# Create a DataFrame to list anomalies
anomalous_data_zscore = df_hist[anomalies.any(axis=1)]

# Display the first few rows of detected anomalies
anomalous_data_zscore.head()

## 1.2 LSTM-based Autoencoders

In [ ]:
# Select only the time series features for LSTM training
time_series_columns = [
    'temperature_2m', 'relative_humidity_2m', 'dew_point_2m', 'rain', 'cloud_cover_low', 'cloud_cover_mid', 
    'et0_fao_evapotranspiration', 'vapour_pressure_deficit', 'wind_speed_10m', 'wind_speed_100m', 'wind_gusts_10m'
]

# Prepare the data in sequences for LSTM
sequence_length = 24  # e.g., a 24-hour window for each sequence
data = df_hist[time_series_columns].values

# Reshape into sequences (num_sequences, sequence_length, num_features)
num_sequences = data.shape[0] // sequence_length
X = data[:num_sequences * sequence_length].reshape(num_sequences, sequence_length, len(time_series_columns))

# Define the LSTM autoencoder model
model = Sequential([
    LSTM(64, activation='relu', input_shape=(X.shape[1], X.shape[2]), return_sequences=True),
    LSTM(32, activation='relu', return_sequences=False),
    RepeatVector(X.shape[1]),
    LSTM(32, activation='relu', return_sequences=True),
    LSTM(64, activation='relu', return_sequences=True),
    TimeDistributed(Dense(X.shape[2]))
])

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model on normal data (use only normal data if you know it beforehand)
history = model.fit(X, X, epochs=50, batch_size=32, validation_split=0.1, shuffle=True)

# Compute reconstruction errors
X_pred = model.predict(X)
mse = np.mean(np.power(X - X_pred, 2), axis=(1, 2))

# Set an anomaly threshold (e.g., based on 99th percentile of MSEs)
threshold = np.percentile(mse, 99)

# Flag sequences as anomalies if the MSE is above the threshold
anomalies = mse > threshold

# Print a few examples of detected anomalies
anomalous_sequences = X[anomalies]
print("Number of anomalies detected:", np.sum(anomalies))

In [ ]:
class LSTMAutoencoderAnomalyDetector:
    def __init__(self, sequence_length=24, lstm_units=[64, 32], epochs=50, batch_size=32, threshold_percentile=99):
        self.sequence_length = sequence_length
        self.lstm_units = lstm_units
        self.epochs = epochs
        self.batch_size = batch_size
        self.threshold_percentile = threshold_percentile
        self.model = None
        # self.scaler = MinMaxScaler()
        self.threshold = None

    def preprocess_data(self, df, time_series_columns):
        # Scale data
        # data = self.scaler.fit_transform(df[time_series_columns])
        data = df[time_series_columns].values
        
        # Reshape into sequences
        num_sequences = data.shape[0] // self.sequence_length
        X = data[:num_sequences * self.sequence_length].reshape(num_sequences, self.sequence_length, len(time_series_columns))
        
        return X

    def build_model(self, input_shape):
        # Build LSTM Autoencoder
        model = Sequential([
            LSTM(self.lstm_units[0], activation='relu', input_shape=input_shape, return_sequences=True),
            LSTM(self.lstm_units[1], activation='relu', return_sequences=False),
            RepeatVector(input_shape[0]),
            
            LSTM(self.lstm_units[1], activation='relu', return_sequences=True),
            LSTM(self.lstm_units[0], activation='relu', return_sequences=True),
            TimeDistributed(Dense(input_shape[1]))
        ])
        model.compile(optimizer='adam', loss='mse')
        self.model = model

    def train(self, X):
        # Train the autoencoder model
        history = self.model.fit(X, X, epochs=self.epochs, batch_size=self.batch_size, validation_split=0.1, shuffle=True)
        return history

    def compute_reconstruction_error(self, X):
        # Predict and calculate reconstruction error
        X_pred = self.model.predict(X)
        mse = np.mean(np.power(X - X_pred, 2), axis=(1, 2))
        return mse

    def set_threshold(self, mse):
        # Set anomaly threshold based on the specified percentile
        self.threshold = np.percentile(mse, self.threshold_percentile)
        
    def detect_anomalies(self, mse):
        # Detect anomalies based on reconstruction error and threshold
        return mse > self.threshold

    def fit_predict(self, df, time_series_columns):
        # Full pipeline from preprocessing, training, and detecting anomalies
        X = self.preprocess_data(df, time_series_columns)
        input_shape = (X.shape[1], X.shape[2])
        
        # Build, train model and compute reconstruction error
        self.build_model(input_shape)
        self.train(X)
        mse = self.compute_reconstruction_error(X)
        
        # Set threshold and detect anomalies
        self.set_threshold(mse)
        anomalies = self.detect_anomalies(mse)
        
        return X[anomalies], np.sum(anomalies)  # Return anomalous sequences and count

# Define columns to be used
time_series_columns = [
    'temperature_2m', 'relative_humidity_2m', 'dew_point_2m', 'rain', 'cloud_cover_low', 
    'cloud_cover_mid', 'et0_fao_evapotranspiration', 'vapour_pressure_deficit', 
    'wind_speed_10m', 'wind_speed_100m', 'wind_gusts_10m'
]

# Instantiate the anomaly detector class
anomaly_detector = LSTMAutoencoderAnomalyDetector(sequence_length=24, epochs=50, batch_size=32)

# Detect anomalies in data
anomalous_sequences, num_anomalies = anomaly_detector.fit_predict(df_hist, time_series_columns)
print("Number of anomalies detected:", num_anomalies)

# for events like Cloudbursts (CB) & Mini-Cloudbursts (MCB) based on periods with rainfall intensities.

1. **Heavy Rainfall**: Rainfall > *6.5 cm/day*
2. **Very Heavy Rainfall**: Rainfall > *13.0 cm/day*
3. **Extremely Heavy Rainfall**: Rainfall > *20.0 cm/day*
3. **Cloudburst (CB)**: Rainfall > *10 cm in a single hour*
4. **Mini-Cloud Burst (MCB)**: Rainfall exceeds *5 cm over any two consecutive hours*